# init

## imports

In [27]:
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

## configs

In [30]:
DATA_PATH = os.path.expanduser('~/class/new_york/data/.data/')

START_DATE = pd.Timestamp('2023-01-01').date()
END_DATE = pd.Timestamp('2023-04-30').date()

POPULAR_AREA_COUNT = 50
LAG_DAYS_COUNT = 10 

OUTPUT_DATASET_PATH = os.path.expanduser('~/class/new_york/data/dataset.csv')

# load data

In [13]:
rides_df = pd.read_parquet(DATA_PATH)
print(rides_df.columns)
rides_df

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12672732,2,2023-04-30 23:50:19,2023-05-01 00:06:11,NaN,2.02,NaN,None,246,79,0,11.41,0.00,0.5,3.08,0.0,1.0,18.49,NaN,NaN
12672733,2,2023-04-30 23:28:32,2023-04-30 23:38:32,NaN,2.36,NaN,None,114,68,0,13.19,0.00,0.5,0.00,0.0,1.0,17.19,NaN,NaN
12672734,2,2023-04-30 23:25:26,2023-04-30 23:44:16,NaN,4.64,NaN,None,137,74,0,17.90,0.00,0.5,0.00,0.0,1.0,21.90,NaN,NaN
12672735,1,2023-04-30 23:11:59,2023-04-30 23:26:19,NaN,0.00,NaN,None,162,151,0,16.00,1.00,0.5,4.20,0.0,1.0,27.20,NaN,NaN


In [52]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,NaN
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,NaN
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,NaN
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,NaN
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288245,2,2023-04-30 23:50:19,2023-05-01 00:06:11,NaN,2.02,NaN,None,246,79,0,11.41,0.00,0.5,3.08,0.0,1.0,18.49,NaN,NaN,NaN
3288246,2,2023-04-30 23:28:32,2023-04-30 23:38:32,NaN,2.36,NaN,None,114,68,0,13.19,0.00,0.5,0.00,0.0,1.0,17.19,NaN,NaN,NaN
3288247,2,2023-04-30 23:25:26,2023-04-30 23:44:16,NaN,4.64,NaN,None,137,74,0,17.90,0.00,0.5,0.00,0.0,1.0,21.90,NaN,NaN,NaN
3288248,1,2023-04-30 23:11:59,2023-04-30 23:26:19,NaN,0.00,NaN,None,162,151,0,16.00,1.00,0.5,4.20,0.0,1.0,27.20,NaN,NaN,NaN


In [53]:
df.iloc[0]

VendorID                                   2
tpep_pickup_datetime     2023-01-01 00:32:10
tpep_dropoff_datetime    2023-01-01 00:40:36
passenger_count                          1.0
trip_distance                           0.97
RatecodeID                               1.0
store_and_fwd_flag                         N
PULocationID                             161
DOLocationID                             141
payment_type                               2
fare_amount                              9.3
extra                                    1.0
mta_tax                                  0.5
tip_amount                               0.0
tolls_amount                             0.0
improvement_surcharge                    1.0
total_amount                            14.3
congestion_surcharge                     2.5
airport_fee                              0.0
Airport_fee                              NaN
Name: 0, dtype: object

# data preprocessing

## filter by date

In [14]:
rides_df['Date'] = pd.to_datetime(rides_df['tpep_pickup_datetime']).dt.date
rides_df= rides_df[rides_df['Date'] >= START_DATE]
rides_df = rides_df[END_DATE >= rides_df['Date']]
rides_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Date
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,2023-01-01
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,2023-01-01
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,2023-01-01
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,2023-01-01
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12672732,2,2023-04-30 23:50:19,2023-05-01 00:06:11,NaN,2.02,NaN,None,246,79,0,11.41,0.00,0.5,3.08,0.0,1.0,18.49,NaN,NaN,2023-04-30
12672733,2,2023-04-30 23:28:32,2023-04-30 23:38:32,NaN,2.36,NaN,None,114,68,0,13.19,0.00,0.5,0.00,0.0,1.0,17.19,NaN,NaN,2023-04-30
12672734,2,2023-04-30 23:25:26,2023-04-30 23:44:16,NaN,4.64,NaN,None,137,74,0,17.90,0.00,0.5,0.00,0.0,1.0,21.90,NaN,NaN,2023-04-30
12672735,1,2023-04-30 23:11:59,2023-04-30 23:26:19,NaN,0.00,NaN,None,162,151,0,16.00,1.00,0.5,4.20,0.0,1.0,27.20,NaN,NaN,2023-04-30


## defining popular areas

In [19]:
pop_area_df = (
    rides_df
    .groupby('PULocationID')
    ['VendorID']
    .count()
    .sort_values(ascending=False)
    .head(POPULAR_AREA_COUNT)
    .reset_index(name='count')
)
popular_areas_list = pop_area_df['PULocationID'].tolist()
pop_area_df

,PULocationID,count
0,132,635312
1,237,590734
2,161,580792
3,236,542459
4,162,444517
5,186,435413
6,230,423159
7,142,416781
8,138,401185
9,170,371145


## **labelling** - trips per area and date

In [21]:
daily_trips_df = (
    rides_df
    .groupby(['Date', 'PULocationID'])
    .size()
    .reset_index(name='Daily_trips')
)

daily_trips_df

,Date,PULocationID,Daily_trips
0,2023-01-01,1,40
1,2023-01-01,4,174
2,2023-01-01,5,3
3,2023-01-01,6,1
4,2023-01-01,7,126
...,...,...,...
26544,2023-04-30,261,440
26545,2023-04-30,262,1214
26546,2023-04-30,263,2189
26547,2023-04-30,264,1007


## filter popular areas daily trips

In [23]:
pop_areas_trips_df = daily_trips_df[daily_trips_df['PULocationID'].isin(popular_areas_list)]
pop_areas_trips_df

,Date,PULocationID,Daily_trips
9,2023-01-01,13,398
18,2023-01-01,24,220
34,2023-01-01,43,1129
38,2023-01-01,48,2894
40,2023-01-01,50,868
...,...,...,...
26536,2023-04-30,249,2490
26544,2023-04-30,261,440
26545,2023-04-30,262,1214
26546,2023-04-30,263,2189


## **feature extraction** - lag daily trips

In [29]:
for lag in range(1, LAG_DAYS_COUNT+1):
    pop_areas_trips_df[f'{lag}_day_lag'] = (
        pop_areas_trips_df
        .groupby('PULocationID')['Daily_trips']
        .rolling(window=lag+1)
        .apply(lambda x: x.iloc[0] if len(x) >= 1 else None)
        .reset_index(level=0, drop=True)
    )
pop_areas_trips_df.dropna(inplace=True)
pop_areas_trips_df

,Date,PULocationID,Daily_trips,1_day_lag,2_day_lag,3_day_lag,4_day_lag,5_day_lag,6_day_lag,7_day_lag,8_day_lag,9_day_lag,10_day_lag
2216,2023-01-11,13,683,633.0,456.0,278.0,417.0,507.0,614.0,493.0,446.0,381.0,398.0
2227,2023-01-11,24,300,271.0,239.0,225.0,320.0,306.0,288.0,277.0,261.0,169.0,220.0
2243,2023-01-11,43,1549,1550.0,1376.0,1340.0,1919.0,1994.0,1859.0,1464.0,1474.0,1283.0,1129.0
2246,2023-01-11,48,2638,2501.0,2039.0,2725.0,3232.0,2787.0,2632.0,2360.0,2053.0,1624.0,2894.0
2248,2023-01-11,50,617,553.0,464.0,578.0,651.0,628.0,532.0,491.0,502.0,447.0,868.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26536,2023-04-30,249,2490,3530.0,2734.0,2363.0,2081.0,1774.0,1452.0,2419.0,3201.0,2592.0,2296.0
26544,2023-04-30,261,440,481.0,551.0,583.0,598.0,497.0,441.0,477.0,611.0,570.0,663.0
26545,2023-04-30,262,1214,1445.0,1785.0,1693.0,1735.0,1579.0,1465.0,1221.0,1451.0,1611.0,1795.0
26546,2023-04-30,263,2189,2671.0,2561.0,2178.0,2069.0,1932.0,1735.0,1970.0,2957.0,2464.0,2262.0


## save output

In [31]:
pop_areas_trips_df.to_csv(OUTPUT_DATASET_PATH, index=False)